# Imports

In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor

import os
import gc

train_path = r'../input/tabular-playground-series-aug-2021/train.csv'
test_path = r'../input/tabular-playground-series-aug-2021/test.csv'
submission_path = r'../input/tabular-playground-series-aug-2021/sample_submission.csv'
RS = 69420
os.environ['NUMEXPR_MAX_THREADS'] = '-1'

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
train = pd.read_csv(train_path, index_col=0)
train.head()

In [ ]:
gc.collect()

In [ ]:
label = 'loss'
eval_metric='rmse'
save_path = r'./AutoGlonAugust/'
timeLimit = 3600 * 7.5
search_strategy = 'auto'
num_trials = 5000

In [ ]:
hyperparameter_kwargs = {
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy
}

In [ ]:
predictor = TabularPredictor(
    label=label,
    problem_type='regression',
    eval_metric=eval_metric,
    path=save_path,
    verbosity=2
)

In [ ]:
%%time
predictor.fit(
    train,
    presets='best_quality',
    keep_only_best=True,
    refit_full=True,
    ag_args_fit={'num_gpus': 1},
    save_space=True,
    hyperparameter_tune_kwargs=hyperparameter_kwargs,
    time_limit=timeLimit
)

In [ ]:
results = predictor.fit_summary()

# Predict Model

In [ ]:
submission = pd.read_csv(submission_path, index_col=0)
submission.head()

In [ ]:
test = pd.read_csv(test_path, index_col=0)
test.head()

In [ ]:
y_preds = predictor.predict_proba(test)

In [ ]:
submission.loss = y_preds
submission.head()

In [ ]:
submission.to_csv('submission.csv')